# Summary
We map cropland across Africa.
1. Set up environment
2. Import AOI
3. Sentinel-2
- Create a cloudless Sentinel-2 collection
- Create periods per year and create features
4. Sentinel-1
- Import Sentinel-1 and create features
6. Segment pixels
7. Train random forest model and classify pixels

### Future
- Explore different phenological measures
- S1 samples have many missing values
- Include more than just temporal NDVI for S2
- Now getting EEException: User memory limit exceeded with S2 bands

# Set up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.environ['GOOGLE_SERVICE_ACCOUNT'] = '[gee-1-238@nature-watch-387210.iam.gserviceaccount.com](mailto:gee-1-238@nature-watch-387210.iam.gserviceaccount.com)'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/keys/nature-watch-keys/nature-watch-gee-1.json'


In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

# Import AOI

In [ ]:
with open("/content/drive/MyDrive/data/cropland/central_sample_aoi.geojson") as f:
    json_data_aoi = json.load(f)

aoi = geemap.geojson_to_ee(json_data_aoi)

with open("/content/drive/MyDrive/data/cropland/central_sample_crop.geojson") as f:
    json_data_crop_aoi = json.load(f)

cropland_aoi = geemap.geojson_to_ee(json_data_crop_aoi)


# Create a cloudless Sentinel-2 collection

In [ ]:
CLOUD_FILTER = 80
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


def get_s2_sr_cld_col(datefilter):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filter(datefilter)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filter(datefilter)
        )

    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


date_filter = ee.Filter.date(ee.Date('2015-12-31'), ee.Date('2023-01-01'))

sentinel = get_s2_sr_cld_col(date_filter).map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

sentinel_vis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}

sentinel_clip = sentinel.filterBounds(aoi)

# Create periods per year

In [ ]:
bands = ['NDVI', 'B5', 'B6', 'B7', 'B8A']

# Calculate NDVI and clip to aoi
def calculate_ndvi(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI').clip(aoi)
    return img.addBands(ndvi)

sentinel_ndvi = sentinel_clip.map(calculate_ndvi).select(bands)



In [ ]:
# Function to create median periods

def compute_yearly_median(img_collection, year_start_date, year_end_date, days):
    num_periods = year_end_date.difference(year_start_date, 'day').divide(days).floor()
    date_sequence = ee.List.sequence(0, num_periods.subtract(1))

    def compute_date(n):
        n = ee.Number(n)
        return year_start_date.advance(n.multiply(days), 'day')

    dates_list = date_sequence.map(compute_date)

    def create_medians(image_index):
        image_index = ee.List(image_index)
        this_date = ee.Date(image_index.get(0))
        period = ee.Number(image_index.get(1))
        median_img = img_collection.filterDate(this_date, this_date.advance(days, 'day')).median()
        return median_img.set('period', period)

    return ee.ImageCollection(ee.List(dates_list.zip(date_sequence)).map(create_medians))



In [ ]:
def add_period_property(img_collection):
  num_images = img_collection.size()
  sequence = ee.List.sequence(0, num_images.subtract(1))

  def add_counter(image_index):
      image_index = ee.List(image_index)
      image = ee.Image(image_index.get(0))
      counter = ee.Number(image_index.get(1))
      return image.set('counter', counter)

  new_image_collection = ee.ImageCollection(ee.List(img_collection.zip(sequence)).map(add_counter))
  return new_image_collection

In [ ]:
# Compute the periods
days = 30

y_start = ee.Date('2022-01-01')
y_end = ee.Date('2023-01-01')
y1_start = y_start.advance(-1, 'year')
y1_end = y_end.advance(-1, 'year')
y2_start = y_start.advance(-2, 'year')
y2_end = y_end.advance(-2, 'year')

y_periods = compute_yearly_median(sentinel_ndvi, y_start, y_end, days)
y1_periods = compute_yearly_median(sentinel_ndvi, y1_start, y1_end, days)
y2_periods = compute_yearly_median(sentinel_ndvi, y2_start, y2_end, days)

In [ ]:
def replace_masked_with_previous_year(feature):
    primary = ee.Image(feature.get('primary'))
    secondary = ee.Image(feature.get('secondary'))
    result = secondary.blend(primary)
    return result


In [ ]:
def join_images(feature):
    primary = ee.Image(feature.get('primary'))
    secondary = ee.Image(feature.get('secondary'))
    combined_image = ee.Image.cat(primary, secondary)

    return combined_image


# YEAR 1: Join images
filter_eq = ee.Filter.equals(leftField='period', rightField='period')
inner_join = ee.Join.inner()
inner_joined_1 = inner_join.apply(y_periods, y1_periods, filter_eq)

# YEAR 1: Map the function over the image collection
backed_filled_1 = ee.ImageCollection(inner_joined_1.map(replace_masked_with_previous_year))


# YEAR 2: Join images
filter_eq = ee.Filter.equals(leftField='period', rightField='period')
inner_join = ee.Join.inner()
inner_joined_2 = inner_join.apply(backed_filled_1, y2_periods, filter_eq)

# YEAR 2: Map the function over the image collection
backed_filled_2 = ee.ImageCollection(inner_joined_2.map(replace_masked_with_previous_year))

In [ ]:
time_img_y = y_periods.select('NDVI').toBands().clip(aoi)
time_img_y1 = y1_periods.select('NDVI').toBands().clip(aoi)
time_img_filled = backed_filled_2.select('NDVI').toBands().clip(aoi).rename(time_img_y.bandNames())


# Import AOI

In [ ]:
print(time_img_filled.bandNames().getInfo())

In [ ]:
bands = ['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI']
image_aoi = time_img_filled.select(bands)

# Sentinel-2 metrics

In [ ]:
# sentinel_bands = ['B5', 'B6', 'B7', 'B8A'] # red edge

# Red-edge slope
def calc_slope(img):
    slope = img.select('B7').subtract(img.select('B5')).divide(ee.Number(835.1-703.9)).rename('slope')
    return img.addBands(slope)

sentinel_slope = backed_filled_2.map(calc_slope)
sentinel_bands = ['slope'] # red edge
red_edge_img = sentinel_slope.select(sentinel_bands).median()


# Forest height
height = ee.ImageCollection('users/potapovpeter/GEDI_V27').median().select('b1').rename('height')
sentinel_max = backed_filled_2.select('NDVI').max().rename('max')
sentinel_min = backed_filled_2.select('NDVI').min().rename('min')



# Sentinel-1 metrics


In [ ]:
# Add VV-VH ratio
def add_ratio(image):
    ratio = image.select('VV').divide(image.select('VH')).rename('VV_VH_ratio')
    return image.addBands(ratio)

# Load Sentinel-1 data
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                .filterBounds(aoi)
                .filterDate('2022-01-01', '2023-01-01')
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.eq('instrumentMode', 'IW'))).map(add_ratio).select(['VV', 'VH', 'VV_VH_ratio'])


sentinel1 = collection.mean().select(['VV', 'VH', 'VV_VH_ratio'])

In [ ]:
# Compute Sentinel-1 periods
days = 15

y_periods_s1 = compute_yearly_median(collection, y_start, y_end, days)
time_img_s1 = y_periods_s1.select('VV_VH_ratio').toBands().clip(aoi)


In [ ]:
print(time_img_s1.bandNames().getInfo())

# Add metrics

In [ ]:
image_bands = image_aoi.addBands(sentinel_max).addBands(sentinel_min).addBands(red_edge_img)

# Segmentation

In [ ]:
snic = ee.Algorithms.Image.Segmentation.SNIC(
    image = image_bands,
    size = 30,
    compactness = 0.1,
    connectivity = 8,
)
print(snic.bandNames().getInfo())


In [ ]:
# snic_bands = ['0_VV_VH_ratio_mean', '1_VV_VH_ratio_mean', '2_VV_VH_ratio_mean', '3_VV_VH_ratio_mean', '4_VV_VH_ratio_mean', '5_VV_VH_ratio_mean', '6_VV_VH_ratio_mean', '7_VV_VH_ratio_mean', '8_VV_VH_ratio_mean', '9_VV_VH_ratio_mean', '10_VV_VH_ratio_mean', '11_VV_VH_ratio_mean', '12_VV_VH_ratio_mean', '13_VV_VH_ratio_mean', '14_VV_VH_ratio_mean', '15_VV_VH_ratio_mean', '16_VV_VH_ratio_mean', '17_VV_VH_ratio_mean']

snic_bands = ['0_NDVI_mean', '1_NDVI_mean', '2_NDVI_mean', '3_NDVI_mean', '4_NDVI_mean', '5_NDVI_mean', '6_NDVI_mean', '7_NDVI_mean', '8_NDVI_mean', '9_NDVI_mean', '10_NDVI_mean', '11_NDVI_mean', 'max_mean', 'min_mean', 'slope_mean']


snic_fixed = snic.reproject(crs='EPSG:4326', scale=2).reproject(crs='EPSG:4326', scale=10).select(snic_bands)

# First export

In [ ]:
# Or manually define AOI

maoi = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.428439235334508,
              6.498584609525565
            ],
            [
              8.428439235334508,
              -8.15166839720959
            ],
            [
              31.680359370309816,
              -8.15166839720959
            ],
            [
              31.680359370309816,
              6.498584609525565
            ],
            [
              8.428439235334508,
              6.498584609525565
            ]
          ]
        ],
      })



In [ ]:
image_name = 'crop_snic_2022_congo'
fileNamePrefix = 'temp/' + image_name

exportConfig = {
    'image': snic_fixed.toFloat(),
    'description': image_name,
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': fileNamePrefix,
    'scale': 30,
    'maxPixels': 4787560500,
    'region': maoi,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
}

task = ee.batch.Export.image.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

# Import segments

In [ ]:
from google.cloud import storage
def list_blobs(bucket_name):
    storage_client = storage.Client.from_service_account_json("/content/drive/MyDrive/keys/nature-watch-keys/nature-watch-387210.json")

    blobs = storage_client.list_blobs(bucket_name, prefix='temp/')
    blob_names = []

    for blob in blobs:
        if blob.name.startswith('temp/crop_snic_2022'):
          blob_names.append('gs://' + bucket_name +'/' + blob.name)

    return blob_names

In [ ]:
segment_blobs = list_blobs('nature-watch-bucket')
segments = geemap.load_GeoTIFFs(segment_blobs)
segment = segments.mosaic()

In [ ]:
print(segment.bandNames().getInfo())

# Sample points

In [ ]:
class_img = snic_fixed

# Randomly split the data into 70% training and 30% validation
cropland_split = cropland_aoi.randomColumn(seed=5)
training = cropland_split.filter(ee.Filter.lt('random', 0.7))
validation = cropland_split.filter(ee.Filter.gte('random', 0.7))

# Sample the pixel values
training_sampled = class_img.sampleRegions(collection=training, properties=['Class'], scale=30, tileScale=13)
validation_sampled = class_img.sampleRegions(collection=validation, properties=['Class'], scale=30, tileScale=13)

In [ ]:
# Classify
classifier = ee.Classifier.smileRandomForest(10).train(features=training_sampled, classProperty='Class', inputProperties=class_img.bandNames())
classified = class_img.classify(classifier)
# confusionMatrix.getInfo()

In [ ]:
# OR classify with probabilities
classifier_p = ee.Classifier.smileRandomForest(10).train(features=training_sampled, classProperty='Class', inputProperties=class_img.bandNames()).setOutputMode('PROBABILITY')
classified_p = class_img.classify(classifier_p)


In [ ]:
validated = validation_sampled.classify(classifier)
confusionMatrix = validated.errorMatrix('Class', 'classification')

overallAccuracy = confusionMatrix.accuracy()

producersAccuracy = confusionMatrix.producersAccuracy() # Sensitivity (recall)
consumersAccuracy = confusionMatrix.consumersAccuracy() # Specificity (precision)

print('Accuracy')
print(overallAccuracy.getInfo())

# print('Sensitivity')
# print(producersAccuracy.getInfo())

# print('Specificity')
# print(consumersAccuracy.getInfo())

In [ ]:
# Feature Importances
importances = classifier.explain().get('importance')

importances_dict = ee.Dictionary(importances).getInfo()

sorted_importances = sorted(importances_dict.items(), key=lambda x: x[1], reverse=True)
for band, importance in sorted_importances:
    print(f'{band}: {importance}')


In [ ]:
cropland = classified.eq(1).selfMask()
cropland_p = classified_p.gt(0.6)

In [ ]:
print(classified.getInfo())

# Export features

In [ ]:
feature_collection_name = 'centralsample_2022_crop_VH'
fileNamePrefix = 'tables/' + feature_collection_name

exportConfig = {
    'collection': training_sampled,
    'description': feature_collection_name,
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': fileNamePrefix,
    'fileFormat': 'CSV'
}

task = ee.batch.Export.table.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

# Compute pixels

In [ ]:
# Region of interest.
coords = [
    22.941, 4.775
]


# Make a projection to discover the scale in degrees.
proj = ee.Projection('EPSG:4326').atScale(10).getInfo()

# Get scales out of the transform.
scale_x = proj['transform'][0]
scale_y = -proj['transform'][4]

# Make a request object.
request = {
    'expression': cropland,
    'fileFormat': 'PNG',
    'bandIds': ['classification'],
    'grid': {
        'dimensions': {
            'width': 640,
            'height': 640
        },
        'affineTransform': {
            'scaleX': scale_x,
            'shearX': 0,
            'translateX': coords[0],
            'shearY': 0,
            'scaleY': scale_y,
            'translateY': coords[1]
        },
        'crsCode': proj['crs'],
    },
    'visualizationOptions': {'ranges': [{'min': 0, 'max': 3000}]},
}

In [ ]:
image_png = ee.data.computePixels(request)


# Map

In [ ]:
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
sentinel1_vis = {'min': [-20, -20, 0], 'max': [0, 0, 2], 'bands': ['VV', 'VH', 'VV_VH_ratio']}


segment_vis = {'min': [-1, 0, -1], 'max': [1, 20, 1], 'bands': ['0_NDVI_mean', 'slope_mean', '5_NDVI_mean']}

Map = geemap.Map()
Map.add_basemap('SATELLITE')
# Map.addLayer(segment, segment_vis, 'segment')
# Map.addLayer(cropland, {}, 'cropland')
Map.addLayer(cropland, {}, 'cropland')

# Map.addLayer(time_img_y.select('3_NDVI'), ndvi_vis, 'y')
# Map.addLayer(time_img_filled.select('3_NDVI'), ndvi_vis, 'filled_1')
# Map.addLayer(primary.select('NDVI'), ndvi_vis, 'y')
# Map.addLayer(secondary.select('NDVI'), ndvi_vis, 'y1')
# Map.addLayer(combined.select('NDVI'), ndvi_vis, '0_NDVI_new')

# Map.addLayer(sentinel1, sentinel1_vis, 'sentinel1');




Map.setCenter(22.941, 4.775, 12)
Map